# Comentarios

1. Se obtienen los rooms nulos o iguales a cero en primera instancia con expresiones regulares. No se obtienen primero con una media condicionada de la superficie cubierta y el lugar ya que es mas preciso obtenerlos con un dato directo del dataset

 **Pasos a seguir:**
1. Extraer precios y superficie con regex (terminado: refinar los patrones de busqueda para obtener mas datos)
2. Obtener el precio a partir del precio cuadrado y la superficie (terminado : no matcheo nada con las condiciones... falta poner igualmente la imputacion)
3. Obtener el precio de metro cuadrado (dolares y currency) mediante la otra variable y una conversion de moneda
4. Imputar las superficies por medio del precio y el precio por metro cuadrado (terminado: no matcheo nada con las condiciones...falta poner igualmente la imputacion)
5. Tomar placename+state name +cantidad de rooms sacar media condicionada para poner en surface
6. Imputar los rooms de los que pudimos extraer con regex
7. Una vez que imputamos la superficie y el precio, buscar el precio por m2

In [27]:
# Tratar de inputar los precios que faltan por expresiones regulares con la columna description y title
# Eliminar todas filas que todo es nulo (check)
# Checar el tema de las superficies
# Precios: unificar todos los precios a dolares
# Hacer la multiplicacion de price por la superficie
# Imputar superficies nulas por medias condicionadas por barrio o cantidad de rooms

# **Estandarizacion de nombres de mascaras, dataset y demas:**
- Al aplicarle una mascara a un dataset seria conveniente primero definir la mascara y en otra linea de codigo aplicarsela al dataset. Esto para que podamos ver bien que le estamos aplicando y que condiciones tiene esa mascara.
- Ojo con utilizar mascaras o alguna otra variable, dataframe y demas, definida mucho mas arriba. No vaya a ser que le cambiemos el nombre por x motivo y nos ollvidamos que mas abajo la usamos para otra cosa y se rompa todo
- Los nombres para las cosas que sean en ingles, separando las palabras con " _ " y en lo posible que se pueda intuir mediante el nombre a que se refiere
- Como practica totalmente arbitraria intentar que los nombres no tengan mas de 5 palabras. ej: " mask_rooms_zeros " , " mask_rooms_surf_zeros " y asi
- Si le llamamos " mask " a algo, que realmente lo sea. No que sea ya una mascara aplicada a un dataset. Eso es el dataset data filtrado por la mascara
- A estos nuevos dataset llamemosle igual que la mascara que le aplicamos solo que reemplazamos el nombre mask por data. ej:  " mask_rooms_zeros "=  " data_rooms_zeros ";                   " data_rooms_surf_zeros " = " data_rooms_surf_zeros "
- Cuando creemos un nuevo dataset, un array, una lista, una serie, mascara o lo que sea poner primero el nombre del tipo de objeto y despues lo que hace, para saber lo que es eso
- Cuando hacemos una mascara para ver si algo es nulo o cero ponemos solo " null ". Es para no alargar los nombres



### Criterios de analisis

- Primero decidimos con que instancias del DataSet vamos a realizar el modelo. En una primera instancia esta decision va a estar basada unicamente en el porcentaje de instancias que representa cada state_name.

# Importacion de librerias utilizadas

In [28]:
import pandas as pd
import numpy as np
import re

# Carga de la data de Properatti como un DataFrame de Pandas

In [29]:
data_location = "../Data/properatti.csv"

data = pd.read_csv(data_location)
data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [30]:
# Eliminamos la columna Unnamed dado que son los mismos indices
data = data.drop(['Unnamed: 0'], axis=1)
data.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [31]:
# Comprobamos la cantidad de filas donde tengan valor nulo en todas las columnas y las eliminamos  
data_clean_empty_row = data.dropna(axis = 0, how= 'all')
print(data.shape)
print(data_clean_empty_row.shape)
#En este caso no hay ninguna fila con todos los valores nulos

(121220, 25)
(121220, 25)


# Limpieza de datos

# Descartado de instancias en funcion del state_name

In [32]:
# Vemos todos los diferentes state_name que hay en el DataSet
data.state_name.unique()

array(['Capital Federal', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Santa Fe', 'Córdoba',
       'Bs.As. G.B.A. Zona Oeste', 'Misiones', 'Buenos Aires Interior',
       'Salta', 'Neuquén', 'Río Negro', 'San Luis', 'Mendoza',
       'Corrientes', 'Chubut', 'Tucumán', 'La Pampa', 'Chaco', 'San Juan',
       'Santa Cruz', 'Tierra Del Fuego', 'Catamarca',
       'Santiago Del Estero', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [33]:
# Vemos la cantidad de instancias que tiene cada state_name
state_name_distribution = pd.value_counts(data.state_name)
state_name_distribution

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

In [34]:
# Vemos los porcentajes de la districion de los datos por provincia
total_data = data.shape[0]
percentage_state_name = 100*state_name_distribution/total_data
percentage_state_name

Capital Federal                 26.658967
Bs.As. G.B.A. Zona Norte        21.085629
Bs.As. G.B.A. Zona Sur          11.509652
Córdoba                          9.956278
Santa Fe                         8.391355
Buenos Aires Costa Atlántica     8.254413
Bs.As. G.B.A. Zona Oeste         7.690150
Buenos Aires Interior            1.889952
Río Negro                        0.666557
Neuquén                          0.604686
Mendoza                          0.561788
Tucumán                          0.556014
Corrientes                       0.480944
Misiones                         0.382775
Entre Ríos                       0.304405
Salta                            0.229335
Chubut                           0.213661
San Luis                         0.207886
La Pampa                         0.129517
Formosa                          0.053622
Chaco                            0.047022
San Juan                         0.032998
Tierra Del Fuego                 0.025573
Catamarca                        0

### Tomamos la decision de desechar las instancias correspondientes a los *state_name* que representan < 1% del total de instancias.

In [35]:
# Tomamos la decision de quedarnos con los datos que son > 1%, que corresponden a Buenos Aires Interior paara arriba
mask_1_percent = percentage_state_name > 1
series_1_percent = percentage_state_name [mask_1_percent]
print(f'La muestra representa el {round(series_1_percent.sum(), 2)}% del total de los datos')
round(series_1_percent, 2)

La muestra representa el 95.44% del total de los datos


Capital Federal                 26.66
Bs.As. G.B.A. Zona Norte        21.09
Bs.As. G.B.A. Zona Sur          11.51
Córdoba                          9.96
Santa Fe                         8.39
Buenos Aires Costa Atlántica     8.25
Bs.As. G.B.A. Zona Oeste         7.69
Buenos Aires Interior            1.89
Name: state_name, dtype: float64

In [36]:
# Con la mascara generada anteriormente para quedarnos con los state_names mayores a 1%, eliminamos las instancias que no corresponden a los state_names seleccionados
data = data[data.state_name.isin(series_1_percent.index)]
data

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,-58.475596,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


# Analis preliminar del DataSet

In [37]:
# Usamos dtypes para saber que tipo de datos se almacenan en el dataset
data.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: object

In [38]:
#Vemos cuantos valores nulos tenemos por columna
is_null_result = data.isnull().sum()
print(is_null_result)

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18527
lat-lon                        48639
lat                            48639
lon                            48639
price                          17870
currency                       17871
price_aprox_local_currency     17870
price_aprox_usd                17870
surface_total_in_m2            37058
surface_covered_in_m2          17744
price_usd_per_m2               49058
price_per_m2                   30175
floor                         108073
rooms                          69989
expenses                      101758
properati_url                      0
description                        2
title                              0
image_thumbnail                 2848
dtype: int64


In [39]:
data_cero = (data==0).sum()
data_cero

operation                       0
property_type                   0
place_name                      0
place_with_parent_names         0
country_name                    0
state_name                      0
geonames_id                     0
lat-lon                         0
lat                             0
lon                             0
price                           1
currency                        0
price_aprox_local_currency      1
price_aprox_usd                 1
surface_total_in_m2           379
surface_covered_in_m2           2
price_usd_per_m2                0
price_per_m2                    0
floor                           0
rooms                           0
expenses                        0
properati_url                   0
description                     0
title                           0
image_thumbnail                 0
dtype: int64

In [40]:
#Vemos que porcentaje de nulos tiene cada columna
percentage_null = 100*is_null_result/data.shape[0]
percentage_null

operation                      0.000000
property_type                  0.000000
place_name                     0.019881
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                   16.014626
lat-lon                       42.043254
lat                           42.043254
lon                           42.043254
price                         15.446719
currency                      15.447583
price_aprox_local_currency    15.446719
price_aprox_usd               15.446719
surface_total_in_m2           32.032709
surface_covered_in_m2         15.337805
price_usd_per_m2              42.405435
price_per_m2                  26.083086
floor                         93.417641
rooms                         60.498064
expenses                      87.958993
properati_url                  0.000000
description                    0.001729
title                          0.000000
image_thumbnail                2.461794


## Correccion de instancias que tienen surface_total_in_m2 < surface_covered_in_m2

In [41]:
# Vemos en que casos el valor surface_total_in_m2 es menor que el valor surface_covered_in_m2
mask_surface_null = data.surface_total_in_m2 < data.surface_covered_in_m2
data_surface_null = data[mask_surface_null]
data_surface_null[["surface_total_in_m2" , "surface_covered_in_m2"]]

,surface_total_in_m2,surface_covered_in_m2
177,420.0,450.0
316,125.0,156.0
317,125.0,156.0
455,170.0,430.0
668,180.0,230.0
...,...,...
120168,120.0,140.0
120181,153.0,226.0
120202,70.0,200.0
120302,1200.0,12000.0


In [42]:
#En los casos en que surface_total_in_m2 sea menor que surface_covered_in_m2 imputamos los datos en el orden inverso

list_a = data_surface_null.surface_total_in_m2
list_b = data_surface_null.surface_covered_in_m2

data.loc[data_surface_null.index, "surface_total_in_m2"] = list_b
data.loc[data_surface_null.index, "surface_covered_in_m2"] = list_a

## Homogeneizacion de precios a moneda dolar

# Obtencion de datos con expresiones regulares de las columnas *description* y *title*

## Obtencion de rooms con expresiones regulares

##### *Se obtienen los rooms nulos o iguales a cero en primera instancia con expresiones regulares. No se obtienen primero con una media condicionada de la superficie cubierta y el lugar ya que es mas preciso obtenerlos con un dato directo del dataset*

### Obtencion de rooms con expresion regular en *description*

In [43]:
# Aplicamos al DataSet una mascara booleana que contenga cero o nulo de rooms
mask_rooms_null = (data.rooms == 0) | (data.rooms.isnull())
data_rooms_null = data [mask_rooms_null]
series_rooms_null_description = data_rooms_null.description
series_rooms_null_description

0         2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...
1         Venta de departamento en décimo piso al frente...
2         2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3         PH 3 ambientes con patio. Hay 3 deptos en lote...
4         DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...
                                ...                        
121215    TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...
121216    Excelente e impecable casa en Venta en Las Lom...
121217    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121218    2 Amb al contrafrente,  luminoso. El departame...
121219    Departamento de 77 m2 apto profesional. Edific...
Name: description, Length: 69989, dtype: object

In [45]:
# Usamos expresiones regulares para sacar los rooms en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras amb indistindo de mayusculas o minusculas
#logica del patron: que tome un numero de un digito que solamente tenga detras uno o mas espacios y seguido de uno o mas espacios y las letras amb
pattern = '(?P<rooms>\d+)\s+(?i)amb' 
pattern_rooms_regex = re.compile(pattern)

In [46]:
# Mostramos donde hay matches
description_rooms_result = series_rooms_null_description.apply(lambda x: pattern_rooms_regex.search(str(x)))
print(description_rooms_result)

0         <re.Match object; span=(0, 5), match='2 AMB'>
1                                                  None
2         <re.Match object; span=(0, 5), match='2 AMB'>
3         <re.Match object; span=(3, 8), match='3 amb'>
4                                                  None
                              ...                      
121215                                             None
121216                                             None
121217                                             None
121218    <re.Match object; span=(0, 5), match='2 Amb'>
121219                                             None
Name: description, Length: 69989, dtype: object


In [47]:
# Extraemos los datos que usaremos para la imputación (pongo que x sea cero cuando es none, ya que si me trae el valor None, no lo puedo pasar a float)
description_rooms_match = description_rooms_result.apply(lambda x: 0 if x is None else x.group('rooms'))
print(description_rooms_match)

print("Rooms matcheados distintos de cero=",(description_rooms_match!=0).sum()) #aca veo que cantidad pudo matchear
print(description_rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

0         2
1         0
2         2
3         3
4         0
         ..
121215    0
121216    0
121217    0
121218    2
121219    0
Name: description, Length: 69989, dtype: object
Rooms matcheados distintos de cero= 18608
['2' 0 '3' '1' '4' '7' '5' '6' '9' '2017' '103' '297' '072' '000' '11' '8'
 '085' '37' '800' '10' '12' '900' '0002' '400' '20174' '006' '17' '90'
 '0001' '772' '2004' '14' '24' '50' '20' '23' '65' '20173']


In [48]:
# Imputamos la serie rooms_match con su index en el DataSet 
data.loc[description_rooms_match.index, "rooms"] = description_rooms_match 
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

       property_type rooms
0                 PH     2
1          apartment     0
2          apartment     2
3                 PH     3
4          apartment     0
...              ...   ...
121215     apartment     0
121216         house     0
121217     apartment     0
121218     apartment     2
121219     apartment     0

[115688 rows x 2 columns]


rooms    object
dtype: object

In [49]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("int")
data[["rooms"]].dtypes
data.rooms.unique() #Aca quiero ver que valores unicos hay de rooms(acompanarlo con un grafico seria lo adecuado)

array([    2,     0,     3,     1,     4,     6,     5,    10,     7,
           9,     8,    17,    22,    15,    12,    11,    14,    16,
          20,    13,  2017,   103,   297,    72,    85,    25,    37,
         800,    19,    30,   900,    18,   400, 20174,    32,    90,
         772,  2004,    24,    50,    23,    31,    21,    29,    27,
          65,    28, 20173])

### Obtencion de rooms con expresion regular en *title*

In [50]:
#Hacemos todo lo mismo que antes pero para obtener los rooms que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_rooms_zero = (data.rooms == 0)
data_rooms_zero = data [mask_rooms_zero]
series_rooms_zero_title = data_rooms_zero.title
series_rooms_zero_title

1         VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
4         DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
6         MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...
7         JOSE HERNANDEZ 1400 MONOAMBIENTE  ESTRENAR CAT...
8         JOSE HERNANDEZ   1400 DOS AMBIENTES ESTRENAR ,...
                                ...                        
121214                                   Local - San Isidro
121215           Torre Forum Alcorta- Impecable 3 ambientes
121216    Ruca Inmuebles | Venta | Lomas de San Isidro |...
121217    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121219           Departamento apto profesional en el Centro
Name: title, Length: 51394, dtype: object

In [51]:
title_rooms_result = series_rooms_zero_title.apply(lambda x: pattern_rooms_regex.search(str(x)))
print(title_rooms_result)

1                                                    None
4          <re.Match object; span=(6, 11), match='2 AMB'>
6                                                    None
7                                                    None
8                                                    None
                               ...                       
121214                                               None
121215    <re.Match object; span=(31, 36), match='3 amb'>
121216                                               None
121217                                               None
121219                                               None
Name: title, Length: 51394, dtype: object


In [52]:
title_rooms_match = title_rooms_result.apply(lambda x: 0 if x is None else x.group('rooms'))
print(title_rooms_match)

print("Rooms matcheados distintos de cero=",(title_rooms_match != 0).sum()) #aca veo que cantidad pudo matchear
print(title_rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

1         0
4         2
6         0
7         0
8         0
         ..
121214    0
121215    3
121216    0
121217    0
121219    0
Name: title, Length: 51394, dtype: object
Rooms matcheados distintos de cero= 2518
[0 '2' '6' '3' '4' '5' '9' '1' '7' '8' '15' '4000' '360' '12']


In [53]:
# Imputamos la serie rooms_match_title con su index en el DataSet 
data.loc[title_rooms_match.index, "rooms"] = title_rooms_match 
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

       property_type rooms
0                 PH     2
1          apartment     0
2          apartment     2
3                 PH     3
4          apartment     2
...              ...   ...
121215     apartment     3
121216         house     0
121217     apartment     0
121218     apartment     2
121219     apartment     0

[115688 rows x 2 columns]


rooms    object
dtype: object

In [54]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("int")
data[["rooms"]].dtypes
data.rooms.unique() #Aca quiero ver que valores unicos hay de rooms(acompanarlo con un grafico seria lo adecuado)

array([    2,     0,     3,     1,     4,     6,     5,    10,     7,
           9,     8,    17,    22,    15,    12,    11,    14,    16,
          20,    13,  2017,   103,   297,    72,    85,    25,    37,
         800,    19,    30,   900,    18,   400, 20174,  4000,    32,
          90,   772,  2004,    24,    50,   360,    23,    31,    21,
          29,    27,    65,    28, 20173])

## Obtencion de surface_total_in_m2 con expresiones regulares

##### *Se obtienen la superficie en la que ambas superficies sean nulas y el precio y el precio por metro cuadrado es nulo o cero. De esta manera sabemos que no podemos imputar una superficie con la otra y no podemos inferir la superficie mediante el precio y el precio por metro cuadrado*

### Obtencion de surface_total_in_m2 con expresion regular en *description*

In [55]:
# Quiero ver que instancias son las que NO voy a poder imputar alguna superficie en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares
# Aplicamos al DataSet una mascara booleana que contenga cero o nulo de surface_covered_in_m2 y surface_total_in_m2 ademas de nulo en price y en price_per_m2 y price_usd_per_m2
surfaces_condition = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
prices_condition = ((data.price == 0) | (data.price.isnull())) & ((data.price_aprox_local_currency == 0) | (data.price_aprox_local_currency.isnull())) & ((data.price_aprox_usd == 0) | (data.price_aprox_usd.isnull()))
prices_per_m2_condition =  ((data.price_usd_per_m2 == 0) | (data.price_usd_per_m2.isnull())) & ((data.price_per_m2 == 0) | (data.price_per_m2.isnull()))

mask_surface_total_in_m2_null = surfaces_condition & (prices_condition | prices_per_m2_condition)
data_surface_total_in_m2_null = data [mask_surface_total_in_m2_null]

series_surface_total_in_m2_null_description = data_surface_total_in_m2_null.description
series_surface_total_in_m2_null_description

data_surface_total_in_m2_null.shape[0]

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en description y title buscando la superficie

11437

In [57]:
# Usamos expresiones regulares para sacar la superficie en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras m2 indistindo de mayusculas o minusculas
pattern = '(?P<surface_total_in_m2>\d+)\s+(?i)m2' 
pattern_surface_total_in_m2_regex = re.compile(pattern)

In [58]:
# Mostramos donde hay matches
description_surface_total_in_m2_result = series_surface_total_in_m2_null_description.apply(lambda x: pattern_surface_total_in_m2_regex.search(str(x)))
print(description_rooms_result)

0         <re.Match object; span=(0, 5), match='2 AMB'>
1                                                  None
2         <re.Match object; span=(0, 5), match='2 AMB'>
3         <re.Match object; span=(3, 8), match='3 amb'>
4                                                  None
                              ...                      
121215                                             None
121216                                             None
121217                                             None
121218    <re.Match object; span=(0, 5), match='2 Amb'>
121219                                             None
Name: description, Length: 69989, dtype: object


In [59]:
description_surface_total_in_m2_match = description_surface_total_in_m2_result.apply(lambda x: 0 if x is None else x.group('surface_total_in_m2'))
print(description_surface_total_in_m2_match)

print("surfaces matcheados distintos de cero=",(description_surface_total_in_m2_match != 0).sum()) #aca veo que cantidad pudo matchear

1           0
3           0
9           0
11          0
65          0
         ... 
121070      0
121079      0
121101      0
121134    124
121144      0
Name: description, Length: 11437, dtype: object
surfaces matcheados distintos de cero= 1235


In [60]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[description_surface_total_in_m2_match.index, "surface_total_in_m2"] = description_surface_total_in_m2_match 
print(data[["property_type" , "surface_total_in_m2"]])
data[["surface_total_in_m2"]].dtypes

       property_type surface_total_in_m2
0                 PH                  55
1          apartment                   0
2          apartment                  55
3                 PH                   0
4          apartment                  35
...              ...                 ...
121215     apartment                 113
121216         house                 360
121217     apartment                  46
121218     apartment                  48
121219     apartment                  77

[115688 rows x 2 columns]


surface_total_in_m2    object
dtype: object

In [61]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["surface_total_in_m2"] = data["surface_total_in_m2"].astype("float")
data[["surface_total_in_m2"]].dtypes

surface_total_in_m2    float64
dtype: object

### Obtencion de surface_total_in_m2 con expresion regular en *title*

In [62]:
#Hacemos todo lo mismo que antes pero para obtener las surface que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_surface_total_in_m2_zero = (data.surface_total_in_m2 == 0)
data_surface_total_in_m2_zero = data [mask_surface_total_in_m2_zero]
series_surface_total_in_m2_zero_title = data_surface_total_in_m2_zero.title
series_surface_total_in_m2_zero_title

1         VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
3                              PH 3 amb. cfte. reciclado   
9                WHITE 7637 - 2 DORMITORIOS CON PATIO      
11                                                  CHALET 
65                                 LOCAL COMERCIAL CON BAÑO
                                ...                        
121069                                   Casa - San Lorenzo
121070                      Barrio Sarmiento - Rondeau 200.
121079                             -Sarmiento- Rondeau 200.
121101    Casa a estrenar en San Matias, 3 dormitorios y...
121144    LA CASA DE TUS SUEÑOS!!! 3 DORM EN SANTINA NOR...
Name: title, Length: 10227, dtype: object

In [63]:
title_surface_total_in_m2_result = series_surface_total_in_m2_zero_title.apply(lambda x: pattern_surface_total_in_m2_regex.search(str(x)))
print(title_surface_total_in_m2_result)

1         None
3         None
9         None
11        None
65        None
          ... 
121069    None
121070    None
121079    None
121101    None
121144    None
Name: title, Length: 10227, dtype: object


In [64]:
title_surface_total_in_m2_match = title_surface_total_in_m2_result.apply(lambda x: 0 if x is None else x.group('surface_total_in_m2'))
print(title_surface_total_in_m2_match)

print("surfaces matcheados distintos de cero=",(title_surface_total_in_m2_match != 0).sum()) #aca veo que cantidad pudo matchear

1         0
3         0
9         0
11        0
65        0
         ..
121069    0
121070    0
121079    0
121101    0
121144    0
Name: title, Length: 10227, dtype: object
surfaces matcheados distintos de cero= 21


In [65]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[title_surface_total_in_m2_match.index, "surface_total_in_m2"] = title_surface_total_in_m2_match 
print(data[["property_type" , "surface_total_in_m2"]])
data[["surface_total_in_m2"]].dtypes

       property_type surface_total_in_m2
0                 PH                  55
1          apartment                   0
2          apartment                  55
3                 PH                   0
4          apartment                  35
...              ...                 ...
121215     apartment                 113
121216         house                 360
121217     apartment                  46
121218     apartment                  48
121219     apartment                  77

[115688 rows x 2 columns]


surface_total_in_m2    object
dtype: object

In [66]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["surface_total_in_m2"] = data["surface_total_in_m2"].astype("float")
data[["surface_total_in_m2"]].dtypes

surface_total_in_m2    float64
dtype: object

## Obtencion de price mediante expresiones regulares

##### *Quiero obtener el price para las instancias en las que no tenemos price y price_aprox_local_currency y price_aprox_usd y tampoco tenemos surface_total_in_m2 y surface_total_in_m2 o no tenemos price_usd_per_m2 y price_per_m2 *

### Obtencion de price con expresion regular en *description*

In [67]:
# Quiero ver que instancias son las que NO voy a poder imputar algun price en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares

surfaces_condition = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
prices_condition = ((data.price == 0) | (data.price.isnull())) & ((data.price_aprox_local_currency == 0) | (data.price_aprox_local_currency.isnull())) & ((data.price_aprox_usd == 0) | (data.price_aprox_usd.isnull()))
prices_per_m2_condition =  ((data.price_usd_per_m2 == 0) | (data.price_usd_per_m2.isnull())) & ((data.price_per_m2 == 0) | (data.price_per_m2.isnull()))


mask_price_null = prices_condition & (prices_per_m2_condition | surfaces_condition)
data_price_null = data [mask_price_null]
series_price_null_description = data_price_null.description
series_price_null_description

data_price_null.shape[0]

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en description y title buscando el price

17871

In [68]:
# Usamos expresiones regulares para sacar la superficie en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras m2 indistindo de mayusculas o minusculas
pattern = '(?P<price>\d+)\s+(?i)usd' 
pattern_price_regex = re.compile(pattern)

C:\Users\gtoso\AppData\Local\Temp\ipykernel_17628\450940615.py:4: DeprecationWarning: Flags not at the start of the expression '(?P<price>\\d+)\\s+(?i' (truncated)
  pattern_price_regex = re.compile(pattern)


In [69]:
# Mostramos donde hay matches
description_price_result = series_price_null_description.apply(lambda x: pattern_price_regex.search(str(x)))
print(description_price_result)

20        None
41        None
62        None
67        None
76        None
          ... 
121210    None
121211    None
121212    None
121213    None
121214    None
Name: description, Length: 17871, dtype: object


In [70]:
description_price_match = description_price_result.apply(lambda x: 0 if x is None else x.group('price'))
print(description_price_match)

print("price matcheados distintos de cero=",(description_price_match != 0).sum()) #aca veo que cantidad pudo matchear

20        0
41        0
62        0
67        0
76        0
         ..
121210    0
121211    0
121212    0
121213    0
121214    0
Name: description, Length: 17871, dtype: object
price matcheados distintos de cero= 11


In [71]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[description_price_match.index, "price"] = description_price_match 
print(data[["property_type" , "price"]])
data[["price"]].dtypes

       property_type   price
0                 PH   62000
1          apartment  150000
2          apartment   72000
3                 PH   95000
4          apartment   64000
...              ...     ...
121215     apartment  870000
121216         house  498000
121217     apartment  131500
121218     apartment   95900
121219     apartment  129000

[115688 rows x 2 columns]


price    object
dtype: object

In [72]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["price"] = data["price"].astype("float")
data[["price"]].dtypes

price    float64
dtype: object

### Obtencion de price con expresion regular en *title*

In [73]:
#Hacemos todo lo mismo que antes pero para obtener las surface que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_price_zero = (data.price == 0)
data_price_zero = data [mask_price_zero]
series_price_zero_title = data_price_zero.title
series_price_zero_title

20        Casa con excelente ubicaciòn, calle asfaltada,...
41        Venta en pozo deptos-dúplex- Barrio Santa Rita...
62                         MODERNA CASA EN OMBUES DE HUDSON
67          Chalet en Venta. Ambientes. Dormitorios. 220 m2
76        MODERNA RESIDENCIA EN STELLA MARIS- s/lote 10x...
                                ...                        
121210                                      Casa - Martinez
121211                              Departamento - Acassuso
121212                                      Casa - Acassuso
121213                                      Casa - Martinez
121214                                   Local - San Isidro
Name: title, Length: 17861, dtype: object

In [74]:
title_price_result = series_price_zero_title.apply(lambda x: pattern_price_regex.search(str(x)))
print(title_price_result)

20        None
41        None
62        None
67        None
76        None
          ... 
121210    None
121211    None
121212    None
121213    None
121214    None
Name: title, Length: 17861, dtype: object


In [75]:
title_price_match = title_price_result.apply(lambda x: 0 if x is None else x.group('price'))
print(title_price_match)

print("price matcheados distintos de cero=",(title_price_match != 0).sum()) #aca veo que cantidad pudo matchear

20        0
41        0
62        0
67        0
76        0
         ..
121210    0
121211    0
121212    0
121213    0
121214    0
Name: title, Length: 17861, dtype: int64
price matcheados distintos de cero= 0


In [76]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[title_price_match.index, "price"] = title_price_match 
print(data[["property_type" , "price"]])
data[["price"]].dtypes

       property_type     price
0                 PH   62000.0
1          apartment  150000.0
2          apartment   72000.0
3                 PH   95000.0
4          apartment   64000.0
...              ...       ...
121215     apartment  870000.0
121216         house  498000.0
121217     apartment  131500.0
121218     apartment   95900.0
121219     apartment  129000.0

[115688 rows x 2 columns]


price    float64
dtype: object

In [77]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["price"] = data["price"].astype("float")
data[["price"]].dtypes

price    float64
dtype: object

# Obtencion de datos mediante imputacion directa

## Obtencion de place_name mediante imputacion directa

In [78]:
#Analizamos los nulos en la columna place_name con una mascara booleana
serie_place_name = data.place_name
mask_place_name_null = serie_place_name.isnull()
data_place_name_null = data[mask_place_name_null]
data_place_name_null

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098,...,240.0,2708.333333,2708.333333,NaN,0,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...
10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067,...,300.0,NaN,1783.333333,NaN,5,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...
11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,-58.626261,...,270.0,253.456221,2037.037037,NaN,0,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...
14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,-58.638174,...,200.0,2000.000000,2400.000000,NaN,0,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...
18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,-58.626350,...,340.0,3814.705882,3814.705882,NaN,0,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...
21922,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.399704,-58.638405",-34.399704,-58.638405,...,220.0,NaN,2000.000000,NaN,5,NaN,http://www.properati.com.ar/17gy5_venta_casa_s...,Corredor Responsable: Matias Castelli - CUCICB...,Venta Casa - Barrio Santa Maria de Tigre,https://thumbs4.properati.com/8/oO7kmFPYNm8DYr...
23664,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4005161998,-58.6417999185",-34.400516,-58.641800,...,294.0,1632.653061,1632.653061,NaN,0,NaN,http://www.properati.com.ar/17mb5_venta_casa_t...,EXCELENTE CASA CLASICA con grandes ventanas y ...,CASA EN VENTA EN SANTA MARIA DE TIGRE CON RENTA,https://thumbs4.properati.com/2/LQCwH6txpFqetn...
24722,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007292,-58.6377783",-34.400729,-58.637778,...,270.0,1677.215190,1962.962963,NaN,0,NaN,http://www.properati.com.ar/17oz1_venta_casa_t...,"EN EXCLUSIVIDAD!!!!!!Lindísima casa, muy lumin...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/0/Nhx4NejZaQACap...
38856,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.425087,-58.5796585",-34.425087,-58.579659,...,550.0,NaN,2454.545455,NaN,0,NaN,http://www.properati.com.ar/18mcb_venta_casa_t...,SANTA MARIA DE TIGRE- AL RIO - ESPECTACULAR CA...,"Casa (barrio cerrado), Venta",https://thumbs4.properati.com/2/Izq8d5o8ti3kJy...
45970,sell,apartment,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.396074,-58.63738",-34.396074,-58.637380,...,27.0,1727.272727,3518.518519,NaN,1,NaN,http://www.properati.com.ar/1958f_venta_depart...,"EDIFICIO ZERENA – EL PALMAR, NORDELTA

#### *Como todos los nulos en la columna place_name corresponden al municipio de Tigre, los imputamos directamente con este municipo*

In [79]:
# Como todos los nulos en la columna place_name corresponden al municipio de Tigre, los imputamos directamente con este municipio
data.loc[data_place_name_null.index, "place_name"] = 'Tigre'

## Obtencion de surface_total_in_m2 y surface_covered_in_m2 mediante imputacion directa

### Datos de Superficie total o superficie cubierta que quedaran nulos o ceros por ahora

In [80]:
# Realizamos una mascara para ver las instancias de superficie tanto cubierta como total de los cuales no tenemos para imputar un valor con el otro

mask_surfaces_null = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
data_surfaces_null = data[mask_surfaces_null]

print (data_surfaces_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#En este DataFrame hay tanto superficies nulas como superficies iguales a cero

       property_type  surface_total_in_m2  surface_covered_in_m2
1          apartment                  0.0                    NaN
3                 PH                  0.0                    NaN
9              house                  0.0                    NaN
11             house                  0.0                    NaN
65             store                  0.0                    NaN
...              ...                  ...                    ...
121069         house                  0.0                    NaN
121070     apartment                  0.0                    NaN
121079     apartment                  0.0                    NaN
121101         house                  0.0                    NaN
121144         house                  0.0                    NaN

[10207 rows x 3 columns]


### Datos de superficie total o superficie que imputo uno con el otro

In [81]:
# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie total pero que la cubierta no es nula ni cero, por lo tanto se la podemos imputar
mask_surface_total_in_m2_null = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))
data_surface_total_in_m2_null = data[mask_surface_total_in_m2_null]

# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie cubierta pero que la total no es nula ni cero, por lo tanto se la podemos imputar
mask_surface_covered_in_m2_null = ((data.surface_total_in_m2 != 0) & (data.surface_total_in_m2.notnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
data_surface_covered_in_m2_null = data[mask_surface_covered_in_m2_null]


print (data_surface_total_in_m2_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])
print (data_surface_covered_in_m2_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])


       property_type  surface_total_in_m2  surface_covered_in_m2
68         apartment                  NaN                   72.0
77         apartment                  NaN                   85.0
79         apartment                  NaN                   42.0
87             house                  NaN                  170.0
89         apartment                  NaN                   53.0
...              ...                  ...                    ...
121155         house                  NaN                  130.0
121156         house                  NaN                  130.0
121207         house                  NaN                  178.0
121208         house                  NaN                  178.0
121210         house                  NaN                  120.0

[26000 rows x 3 columns]
       property_type  surface_total_in_m2  surface_covered_in_m2
10             house               1514.0                    NaN
15             house                164.0                    NaN

In [82]:
# Aca imputamos los valores nulos o ceros de superficie total con los valores de superficie cubierta, utilizando la mascara mask_surface_total_in_m2_zero
data.loc[data_surface_total_in_m2_null.index, "surface_total_in_m2"] = data.loc[data_surface_total_in_m2_null.index, "surface_covered_in_m2"]

# Aca imputamos los valores nulos o ceros de superficie cubierta con los valores de superficie total, utilizando la mascara mask_surface_covered_in_m2_zero
data.loc[data_surface_covered_in_m2_null.index, "surface_covered_in_m2"] = data.loc[data_surface_covered_in_m2_null.index, "surface_total_in_m2"]

### Obtención de las superficias nulas en funcion del precio y del precio por metro cuadrado

##### Vemos instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [83]:
#Aca veo las instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [84]:
# Se hacen tres mascaras booleanas: mask_surfaces_null = instancias sin datos de superficie / mask_prices_notnull = instancias con algun dato de precio / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_null = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
condition_prices_notnull = ((data.price != 0) & (data.price.notnull())) | ((data.price_aprox_local_currency != 0) & (data.price_aprox_local_currency.notnull())) | ((data.price_aprox_usd != 0) & (data.price_aprox_usd.notnull()))
condition_prices_per_m2_notnull = ((data.price_usd_per_m2!=0) & (data.price_usd_per_m2.notnull())) | ((data.price_per_m2!=0) & (data.price_per_m2.notnull()))

mask_surfaces_imput = condition_surfaces_null & condition_prices_notnull & condition_prices_per_m2_notnull
data_surfaces_imput = data[mask_surfaces_imput]

data_surfaces_imput[["price" , "price_aprox_local_currency" , "price_aprox_usd" , "price_usd_per_m2" , "price_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

,price,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,surface_total_in_m2,surface_covered_in_m2


## Obtencion de price mediante imputacion directa en funcion de prices_per_m2 y surface

In [85]:
# Se hacen tres mascaras booleanas: mask_surfaces_nonull = instancias con datos de superficie no nulos / mask_prices_null = instancias sin datos de precios / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_notnull = ((data.surface_total_in_m2 != 0) & (data.surface_total_in_m2.notnull())) | ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))
condition_prices_null = ((data.price == 0) | (data.price.isnull())) & ((data.price_aprox_local_currency == 0) | (data.price_aprox_local_currency.isnull())) & ((data.price_aprox_usd == 0) | (data.price_aprox_usd.isnull()))
condition_prices_per_m2_notnull = ((data.price_usd_per_m2!=0) & (data.price_usd_per_m2.notnull())) | ((data.price_per_m2!=0) & (data.price_per_m2.notnull()))

mask_prices_imput = condition_surfaces_null & condition_prices_notnull & condition_prices_per_m2_notnull
data_prices_imput = data[mask_prices_imput]

data_prices_imput[["price" , "price_aprox_local_currency" , "price_aprox_usd" , "price_usd_per_m2" , "price_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

,price,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,surface_total_in_m2,surface_covered_in_m2


## Obtencion de price_per_m2 mediante imputacion directa en funcion de surface y price

In [86]:
# Se hacen tres mascaras booleanas: mask_surfaces_null = instancias sin datos de superficie / mask_prices_notnull = instancias con algun dato de precio / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_notnull = ((data.surface_total_in_m2 != 0) & (data.surface_total_in_m2.notnull())) | ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))
condition_prices_notnull = ((data.price != 0) & (data.price.notnull())) | ((data.price_aprox_local_currency != 0) & (data.price_aprox_local_currency.notnull())) | ((data.price_aprox_usd != 0) & (data.price_aprox_usd.notnull()))
condition_prices_per_m2_null = ((data.price_usd_per_m2 ==0) | (data.price_usd_per_m2.isnull())) & ((data.price_per_m2 ==0) | (data.price_per_m2.isnull()))

mask_prices_per_m2_imput = condition_surfaces_notnull & condition_prices_notnull & condition_prices_per_m2_null
data_prices_per_m2_imput = data[mask_prices_per_m2_imput]

data_prices_per_m2_imput[["price" , "price_aprox_local_currency" , "price_aprox_usd" , "price_usd_per_m2" , "price_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

,price,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,surface_total_in_m2,surface_covered_in_m2
675,130000.0,2293785.00,130000.00,NaN,NaN,7.0,7.0
951,320000.0,5646240.00,320000.00,NaN,NaN,1600.0,1600.0
952,450000.0,7940025.00,450000.00,NaN,NaN,800.0,800.0
1017,650000.0,643219.30,36454.38,NaN,NaN,150.0,150.0
1402,150000.0,2646675.00,150000.00,NaN,NaN,4.0,4.0
...,...,...,...,...,...,...,...
119502,235000.0,4146457.50,235000.00,NaN,NaN,227.0,227.0
119823,420000.0,415618.66,23555.14,NaN,NaN,400.0,400.0
119835,360000.0,6352020.00,360000.00,NaN,NaN,126.0,126.0
120122,198000.0,3493611.00,198000.00,NaN,NaN,110.0,110.0


In [99]:
data_prices_per_m2_imput.notnull().sum()

operation                     615
property_type                 615
place_name                    615
place_with_parent_names       615
country_name                  615
state_name                    615
geonames_id                   517
lat-lon                       358
lat                           358
lon                           358
price                         615
currency                      607
price_aprox_local_currency    607
price_aprox_usd               607
surface_total_in_m2           615
surface_covered_in_m2         615
price_usd_per_m2                0
price_per_m2                    0
floor                           4
rooms                         615
expenses                       33
properati_url                 615
description                   615
title                         615
image_thumbnail               606
dtype: int64

## En revision: Buscar con una expresion regular imputar la superficie con los datos de 'description' y 'title'

## Analisis de columna rooms

In [87]:
#Aca veo, de los rooms que son iguales a cero, cuales tienen la superficie cubierta distinta de cero o nulo para imputarla despues
data_rooms_surface = data[(data.rooms == 0) & ((data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0) | (data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0))]
data_rooms_surface[["rooms" , "surface_covered_in_m2"]]

,rooms,surface_covered_in_m2
6,0,78.0
7,0,40.0
8,0,60.0
10,0,1514.0
15,0,164.0
...,...,...
121206,0,50.0
121209,0,157.0
121216,0,360.0
121217,0,39.0


# *Analisis columnas de precios*

Hacemos una mascara para nulos dolar y nulos local m2

In [88]:
data_price_null = data[((data.price_usd_per_m2.isnull()) | (data.price_usd_per_m2 == 0)) & ((data.price_per_m2.isnull()) | (data.price_per_m2== 0))]
data_price_null[['price', 'price_usd_per_m2', 'price_per_m2', 'surface_total_in_m2']]
# Estas instancias son las que tendremos que imputar por superficio y precio, si no tiene alguno se tendran que explorar nuevas opciones
print(data_price_null.isnull().sum())
print(data_price_null.shape)

operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                    2813
lat-lon                       11291
lat                           11291
lon                           11291
price                             0
currency                      17871
price_aprox_local_currency    17870
price_aprox_usd               17870
surface_total_in_m2               0
surface_covered_in_m2         10207
price_usd_per_m2              25390
price_per_m2                  25390
floor                         24376
rooms                             0
expenses                      24079
properati_url                     0
description                       1
title                             0
image_thumbnail                1111
dtype: int64
(25390, 25)


In [89]:
data_price_surface_notnull = data_price_null[((data_price_null.price.notnull()) & (data_price_null.price != 0)) & ((data_price_null.surface_total_in_m2.notnull()) & (data_price_null.surface_total_in_m2 != 0))]
data_price_surface_notnull

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
675,sell,house,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-37.9846383,-57.5873841",-37.984638,-57.587384,...,7.0,NaN,NaN,NaN,3,NaN,http://www.properati.com.ar/15flc_venta_casa_m...,VIVINEDA CON LOCAL / CHALET 3 AMB consta de ...,VIVINEDA CON LOCAL / CHALET 3 AMB C/GARAGE DOBLE,https://thumbs4.properati.com/7/mGiz1D0njTP1_0...
951,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,...,1600.0,NaN,NaN,NaN,0,NaN,http://www.properati.com.ar/15gnp_venta_casa_r...,CODIGO: 3063-F 13-01 ubicado en: GARITA 12 SAL...,GARITA 12 SALIDA A 2 CALLES Y RUTA 9 MODERNA ...,https://thumbs4.properati.com/1/hiheKXDWaX1rGn...
952,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,...,800.0,NaN,NaN,NaN,0,NaN,http://www.properati.com.ar/15gnr_venta_casa_r...,CODIGO: 3063-FHM- 05 ubicado en: FUNES HILLS M...,FUNES HILLS MIRAFLORES FABULOSA PROPIEDAD!!! 3...,https://thumbs4.properati.com/1/3xNG1EGlaxDRoz...
1017,sell,house,Bahía Blanca,|Argentina|Buenos Aires Interior|Bahía Blanca|,Argentina,Buenos Aires Interior,3865084.0,"-38.7003698,-62.2046152",-38.700370,-62.204615,...,150.0,NaN,NaN,NaN,5,NaN,http://www.properati.com.ar/15gxa_venta_casa_b...,CODIGO: 3714-114 ubicado en: parera 4500 - Pu...,CASA VENTA NUEVA ESPORA BAHIA BLANCA. CASA EN ...,https://thumbs4.properati.com/7/5vkcWNXPyzGytB...
1402,sell,apartment,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6473774,-58.5191897",-34.647377,-58.519190,...,4.0,NaN,NaN,NaN,3,NaN,http://www.properati.com.ar/15i7f_venta_depart...,CODIGO: ubicado en: Caaguazu - Publicado por...,Departamento de 3 ambientes en pozo,https://thumbs4.properati.com/6/dzoMj-56L1jVPz...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119502,sell,house,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.642034,-58.5181027",-34.642034,-58.518103,...,227.0,NaN,NaN,NaN,0,NaN,http://www.properati.com.ar/1cgqo_venta_casa_l...,CODIGO: 1019-V189 ubicado en: VENTURA BOSCH 64...,"DUPLEX CON PATIO , 227 M2 ,GARAGE , SIN EXPENS...",https://thumbs4.properati.com/8/oyxxhkmHmuTWWt...
119823,sell,house,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,"-31.291839,-64.468714",-31.291839,-64.468714,...,400.0,NaN,NaN,NaN,5,NaN,http://www.properati.com.ar/1ch5n_venta_casa_c...,La propiedad se encuentra enclavada en un terr...,VENTA CASA SANTA MARÍA DE PUNILLA,https://thumbs4.properati.com/0/DhKTAMHHzer2Uc...
119835,sell,store,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,3435874.0,NaN,NaN,NaN,...,126.0,NaN,NaN,NaN,0,3000.0,http://www.properati.com.ar/1ch6d_venta_local_...,CODIGO: 4064-13 ubicado en: av Juan Jose Moren...,Local con Renta Caballito Av Jose M Moreno,https://thumbs4.properati.com/9/pAap6Zjzq02vSD...
120122,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-38.0054923,-57.5525794",-38.005492,-57.552579,...,110.0,NaN,NaN,NaN,3,NaN,http://www.properati.com.ar/1chjn_venta_depart...,APTO CREDITO - Plaza Mitre - Excelente semipis...,Semipiso 3 Amb muy amplio,https://thumbs4.properati.com/9/YOiGR_LmHTrD9v...


In [90]:
data.price[20]

0.0

In [91]:
data.surface_total_in_m2[20]

148.0

# **VENIMOS CODEANDO HASTA ACA, NO DAR BOLA A LO QUE SIGUE**

In [92]:

# Queremos, para los que tengan ambas superficies nulas o cero se las imputamos por una media condicionada por el valor de property_type, place_name, state_name y rooms

In [ ]:
#Aca hago una maquinita manual para obtener el numero de instancia de los valores raros de rooms
marcaradedatos =  data[data.rooms == "0002"]
marcaradedatos.description[43292]

In [ ]:
#Aca empiezo a tomar una muestra para ver que pasa

In [ ]:
data.description[121134     ]

In [ ]:
data.description[1601]

In [ ]:
data.description[116017]

In [ ]:
data.description[76675	]

In [ ]:
data.description[7689]

## Analisis de columna price

### Intentaremos imputar las instancias de precios nulas o iguales a cero con expresiones regulares en la columna descripcion

In [ ]:
data[data.surface_total_in_m2.isnull()]

In [ ]:
data[surfaces_condition]

In [ ]:
mask_prices_surfaces_prices_m2_null = (surfaces_condition & prices_condition) | prices_per_m2_condition
data[mask_prices_surfaces_prices_m2_null]

In [ ]:
prices_condition